In [ ]:
#durveshhhhhhhhhhhh

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib

# Load and encode
df = pd.read_csv(r"C:\Users\Durvesh\Downloads\insurance.csv")
df = pd.get_dummies(df, drop_first=True)

X = df.drop("charges", axis=1)
y = df["charges"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "insurance_model.pkl")


['insurance_model.pkl']

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load or train model
try:
    model = joblib.load("insurance_model.pkl")
except:
    df = pd.read_csv("insurance.csv")
    df = pd.get_dummies(df, drop_first=True)
    X = df.drop("charges", axis=1)
    y = df["charges"]
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    joblib.dump(model, "insurance_model.pkl")

# --- WIDGETS ---
age = widgets.IntSlider(min=18, max=100, value=30, description='Age:')
height = widgets.FloatSlider(min=1.2, max=2.2, step=0.01, value=1.7, description='Height (m):')
weight = widgets.FloatSlider(min=30, max=150, step=1, value=70, description='Weight (kg):')
children = widgets.IntSlider(min=0, max=5, value=1, description='Children:')
sex = widgets.Dropdown(options=['male', 'female'], description='Sex:')
smoker = widgets.Dropdown(options=['yes', 'no'], description='Smoker:')

# Indian states for region
region_map = {
    "Maharashtra": "west",
    "Gujarat": "west",
    "Punjab": "north",
    "Delhi": "north",
    "West Bengal": "east",
    "Assam": "east",
    "Kerala": "south",
    "Tamil Nadu": "south",
    "Karnataka": "south",
    "Uttar Pradesh": "north"
}
region_dropdown = widgets.Dropdown(options=list(region_map.keys()), description='State:')

button = widgets.Button(description="Predict 💰")
output = widgets.Output()

# --- FUNCTION ---
def predict_premium(b):
    output.clear_output()
    
    # BMI calculation
    bmi_value = weight.value / (height.value ** 2)
    if bmi_value < 18.5:
        bmi_category = "Underweight"
    elif bmi_value < 24.9:
        bmi_category = "Normal"
    elif bmi_value < 29.9:
        bmi_category = "Overweight"
    else:
        bmi_category = "Obese"

    # ML feature encoding
    sex_male = 1 if sex.value == "male" else 0
    smoker_yes = 1 if smoker.value == "yes" else 0

    region = region_map[region_dropdown.value]
    region_northwest = 1 if region == "north" else 0
    region_southeast = 1 if region == "south" else 0
    region_southwest = 1 if region == "west" else 0

    features = np.array([[age.value, bmi_value, children.value,
                          sex_male, smoker_yes, region_northwest,
                          region_southeast, region_southwest]])
    
    predicted_cost = model.predict(features)[0]

    # Simulated policy suggestions
    suggestions = {
        "north": ["Max Bupa", "Star Health"],
        "south": ["Niva Bupa", "Religare"],
        "west": ["Care Health", "Tata AIG"],
        "east": ["HDFC Ergo", "ICICI Lombard"]
    }

    expense_coverage = [
        "- Pre/post hospitalization (30/60 days)",
        "- Room rent, ICU, surgeries",
        "- Day care procedures",
        "- AYUSH coverage",
        "- Maternity (some plans)"
    ]

    with output:
        print(f"📏 BMI: {bmi_value:.2f} — {bmi_category}")
        print(f"💰 Estimated Premium: ₹{round(predicted_cost):,}")
        print(f"🏢 Suggested Insurers in {region_dropdown.value}:")
        for comp in suggestions[region]:
            print(f"   ✔ {comp}")
        print("\n🧾 Common Expenses Covered:")
        for item in expense_coverage:
            print(f"   - {item}")

button.on_click(predict_premium)

# --- DISPLAY ---
display(widgets.VBox([
    age, height, weight, children, sex, smoker, region_dropdown,
    button, output
]))
